<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/7B_%E6%8C%87%E6%A8%99%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB_%E9%A0%BB%E5%87%BA%E5%8D%98%E8%AA%9E%E3%81%AE%E8%AA%BF%E6%9F%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>作成した分類モデルを活用してbowで頻出単語の調査</center>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/研究室/研究_M2/src

In [ ]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  #print(DATETIME_now)

  return DATETIME_now

In [ ]:
#TYPE_INDEX='EorI_E'
#TYPE_INDEX='NorS_N'
#TYPE_INDEX='ForT_F'
TYPE_INDEX='JorP_J'

In [ ]:
INPUT_CSV_v7='./Assets/Assets_Output/Tweet_Datasets/v7S1_forBERT_2021-12-04_W20R10000.csv'

In [ ]:
LEANING_RATE=2e-7
BATCH_SIZE=16
EPOCH=20

In [ ]:
SEASONS='S12'
WORDS = 20
RAFFLE=30000

SWITCH_CONTINUE ='off'
#WITCH_CONTINUE ='on'

#model_S10ver2_L2e-07_B16_E20_W20_R30000_EorI_E.tar

In [ ]:
TERM='%s_L%s_B%s_E%s_W%s_R%s' % (SEASONS, LEANING_RATE, BATCH_SIZE, EPOCH, WORDS, RAFFLE)

print(TERM)

S12_L2e-07_B16_E20_W20_R30000


In [ ]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

Defaulting to user installation because normal site-packages is not writeable
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/sh: 1: aptitude: not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable




---


In [ ]:
import pandas as pd
df=pd.read_csv('%s' % INPUT_CSV_v7, engine='python')
# データの確認
print(f'データサイズ： {df.shape}')
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].sample(10)

df.dropna(how='any', axis=0, inplace=True)
print(f'データサイズ： {df.shape}')

df=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df[['text', 'tweet_time', 'MBTI_Type','label_EorI_E','label_NorS_N','label_ForT_F','label_JorP_J']]

データサイズ： (10000, 14)
データサイズ： (10000, 14)


,text,tweet_time,MBTI_Type,label_EorI_E,label_NorS_N,label_ForT_F,label_JorP_J
0,#キャプテン翼新アニメにありそうなこと 日向君の雷獣シュートが意図的にピッチを削る「反スポー...,2017-12-14 15:18:05+00:00,ISFJ,0,0,1,1
1,なるほどご本人のツイート辿ったら、リアル友人が、一対一で話すのに、かつご本人がネタバレから入...,2017-11-01 11:17:14+00:00,INTP,0,1,0,0
2,不要物の処理をそろそろ始めないとものが溜まってきた。 今日と明日で何とかしなくては…,2017-12-09 10:41:53+00:00,ISTJ,0,0,0,1
3,年末年始とお盆は、母と妹の機嫌がものすごーく悪くなるので、居心地が悪い,2017-12-29 13:26:49+00:00,INFJ,0,1,1,1
4,風の噂で夜空を見上げる そこには頬をつたう涙の様な星が ふたご座流星群みたいです(^^),2017-12-13 15:29:30+00:00,INFP,0,1,1,0
...,...,...,...,...,...,...,...
9995,ブキヤの可動フィギュアってキューポッシュくらいやで 今までと同じ素体使うならブキヤを巻き込む...,2017-12-29 04:43:45+00:00,INFJ,0,1,1,1
9996,黒潮12年ぶり大蛇行 漁影響か | 2017/9/29(金) - Yahoo!ニュース,2017-09-29 10:46:07+00:00,ISTJ,0,0,0,1
9997,待ち合わせに１人来ない。 眠いおうち帰りたい んでFF14したい。,2017-10-15 03:28:25+00:00,INFP,0,1,1,0
9998,うわ、これ、西村艦隊ちゃんと育ってないから無理っぽくない…？大丈夫？大丈夫なの？山雲と満潮が...,2017-11-19 22:50:22+00:00,INTP,0,1,0,0


In [ ]:
sentences= df['text'].values #sentences = df.sentence.values
labels= df['label_%s' % TYPE_INDEX].values #labels = df.label.values

## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


from tqdm.notebook import tqdm
import numpy as np
# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)

list_word=[]
list_word.clear()
list_sentence_ifidf=[]
list_sentence_ifidf.clear()
for sent in tqdm(sentences):
    # Tokenizeで分割
    print(sent)
    token_words = tokenizer.tokenize(sent)
    one_sentence=' '.join(token_words)
    list_sentence_ifidf.append(one_sentence)
    # print(sent)
    # print(token_words)
    # print(type(token_words))
    # for element in token_words:
    #   print(element)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
len_max=max(max_len)
#len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
#len_q75=int(len_q75)
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

#キャプテン翼新アニメにありそうなこと 日向君の雷獣シュートが意図的にピッチを削る「反スポーツ的行為」に該当しネットを中心に炎上、翌日謝罪に追い込まれる。
なるほどご本人のツイート辿ったら、リアル友人が、一対一で話すのに、かつご本人がネタバレから入る人であるのもわかってるのに、それでも濁されるから業を煮やしていらっしゃるのか。それならその友人が悪いね
不要物の処理をそろそろ始めないとものが溜まってきた。 今日と明日で何とかしなくては…
年末年始とお盆は、母と妹の機嫌がものすごーく悪くなるので、居心地が悪い
風の噂で夜空を見上げる そこには頬をつたう涙の様な星が ふたご座流星群みたいです(^^)
コミケを前にして本が搬入されてない夢と一冊も売れない夢みたんであと遅刻する夢見ればコンプです
作品のコピー、８割は「良かった」か「人による」で良い気がしてきたよ
赤賀の晴れ着は嬉しいがその前に本体を出すんだよこのタコ助  何回回らせりゃ気が済むんだ… 同じサーバーに完凸させる為に2,500周した猛者もいたが兵は拙速を尊ぶんだから早く寄越せ何でも四万十川
「なんつー石頭やねん｣  ゴリラやのーて、雪男ちゃうんか、あんた？
ご、ごめん　#今後の生でいつかの再開があるかもしれない　#名前も顔も覚えていないけどね…　#うっすらと覚えてるのは袖なしの開襟シャツとハーフパンツの柄さ

['# キャプテン 翼 新 アニメ に あり そう な こと 日向 君 の 雷 ##獣 シュート が 意図 的 に ピッチ を 削 ##る 「 反 スポーツ 的 行為 」 に 該当 し ネット を 中心 に 炎上 、 翌日 謝罪 に 追い込ま れる 。', 'なる ##ほど ご 本人 の ツイート 辿っ たら 、 リアル 友人 が 、 一対 ##一 で 話す のに 、 かつ ご 本人 が ネタ バレ から 入る 人 で ある の も わかっ てる のに 、 それでも 濁 ##さ れる から 業 を 煮 ##やし て いら ##っ ##しゃ ##る の か 。 それ ##なら その 友人 が 悪い ね', '不要 物 の 処理 を そろ ##そ ##ろ 始め ない と もの が 溜 ##まっ て き た 。 今日 と 明日 で 何とか し なく て は ...', '年末年始 と お ##盆 は 、 母 と 

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = len_max,        # 文章の長さを固定（Padding/Trancatinating）#mayo
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                        truncation=True,
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# tenosor型に変換
labels = torch.tensor(labels)

print(input_ids)
print(attention_masks)
print(labels)
# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

/home/20VR016F/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



tensor([[    2,  4931,  9296,  ...,     0,     0,     0],
        [    2,   139,  6867,  ...,     0,     0,     0],
        [    2,  9191,   410,  ...,     0,     0,     0],
        ...,
        [    2,  7325,  2682,  ...,     0,     0,     0],
        [    2,   205, 28544,  ...,     0,     0,     0],
        [    2,  2313,  1179,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([1, 0, 1,  ..., 0, 0, 0])




---



In [ ]:
# for element1, element2, element3 in dataset:
#   print(element1)
#   print(element2)
#   print(element3)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
print("./model_%s_%s.tar" % (TERM, TYPE_INDEX))
checkpoint = torch.load("./model_%s_%s.tar" % (TERM, TYPE_INDEX))
model.load_state_dict(checkpoint["model_state_dict"])

./model_S12_L2e-07_B16_E20_W20_R30000_JorP_J.tar


<All keys matched successfully>

In [ ]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install torchviz

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from tqdm.notebook import tqdm

model.eval()# 訓練モードをオフ

# 検証データローダー
preds_dataloader = DataLoader(
            dataset, 
            sampler = SequentialSampler(dataset), # 順番にデータを取得してバッチ化
            batch_size = 1
        )

list_preds=[]
list_preds.clear()
list_b_labels=[]
list_b_labels.clear()
for batch in tqdm(preds_dataloader):
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  list_b_labels.append(b_labels[0].item())
  #print(len(b_input_ids))
  #print(len(b_input_mask))
  #print(b_labels.values)
  with torch.no_grad():   
      # 学習済みモデルによる予測結果をpredsで取得     
      preds = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
      #print(preds[0].cpu().numpy()[0])
      list_preds.append(preds[0].cpu().numpy()[0])

In [ ]:
# STC ='今日は晴れている。'
# ans = model(STC)
# ans_index=np.argmax(ans[0])
# print('あなたの表情は', classes[ans_index], 'です。')



---



In [ ]:
# print(list_b_labels)
# for idx, element in enumerate(list_b_labels):
#   list_b_labels[idx]=element.item()
# print(list_b_labels)

In [ ]:
list_sentences=[]
list_sentences.clear()
print(sentences)
for idx, element in enumerate(sentences):
  #list_element[idx]=element.item()
  list_sentences.append(element)

['#キャプテン翼新アニメにありそうなこと 日向君の雷獣シュートが意図的にピッチを削る「反スポーツ的行為」に該当しネットを中心に炎上、翌日謝罪に追い込まれる。'
 'なるほどご本人のツイート辿ったら、リアル友人が、一対一で話すのに、かつご本人がネタバレから入る人であるのもわかってるのに、それでも濁されるから業を煮やしていらっしゃるのか。それならその友人が悪いね'
 '不要物の処理をそろそろ始めないとものが溜まってきた。 今日と明日で何とかしなくては…' ...
 '待ち合わせに１人来ない。 眠いおうち帰りたい んでFF14したい。'
 'うわ、これ、西村艦隊ちゃんと育ってないから無理っぽくない…？大丈夫？大丈夫なの？山雲と満潮がぁああああああ＞RT'
 'オリジナルシステム作りたいわけじゃないけど、こういう世界観で遊びたい！ってなっても、それに適したシステムが無いと、オリジナルシステム作るべきなのかなあ？と考えて、やっぱそれはだいぶニッチな感じだから止めようってなることはわりとよくある']


In [ ]:
list_probs=[]
list_probs.clear()
for i in range(0, len(list_preds)):
  list_probs.append(np.nan)
#print(list_probs)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # print(x)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

for idx, element in enumerate(list_preds):
  list_probs[idx]=softmax(element)
  # print()

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd

# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(list_preds, columns=['logit_0', 'logit_1']) #1はEN
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(list_preds, axis=1), columns=['pred_label'])
label_df = pd.DataFrame(list_b_labels, columns=['true_label'])
text_df  = pd.DataFrame(list_sentences, columns=['text'])
probs_df = pd.DataFrame(list_probs, columns=['prob_0', 'prob_1']) #1はEN
accuracy_df = pd.concat([text_df, logits_df, pred_df, label_df, probs_df], axis=1)
# accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()

accuracy_df.to_csv('./V7_%s_%s.csv' % (TERM, TYPE_INDEX))
print(('./V7_%s_%s.csv' % (TERM, TYPE_INDEX)))

./V7_S12_L2e-07_B16_E20_W20_R30000_JorP_J.csv


In [ ]:
df_prob_0 = accuracy_df.sort_values('prob_0', ascending=False)
df_prob_1= accuracy_df.sort_values('prob_1', ascending=False)

In [ ]:
from IPython.display import HTML
#df_prob_0.head(20)
HTML(df_prob_0.head(10).to_html())

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
7984,ああああああっしゃああああ！！！！！ ありがとうございます！！！ありがとうございます！！！！,0.752692,-0.258716,0,0,0.733296,0.266704
7425,ぇえぇえ！！！すごく！すごく！おめでとうございます🎈🎈お幸せに〜〜〜！！！,0.622143,-0.308618,0,0,0.717230,0.282770
6042,あああああああああああああああああああああああああああああああ！！！！！！！！！！！（ﾌﾞﾘﾌﾞﾘﾌﾞﾘﾌﾞﾘｭﾘｭﾘｭﾘｭﾘｭﾘｭ！！！！！！ﾌﾞﾂﾁﾁﾌﾞﾌﾞﾌﾞﾁﾁﾁﾁﾌﾞﾘﾘｲﾘﾌﾞﾌﾞﾌﾞﾌﾞｩｩｩｩｯｯｯ！！！！！！！ ）,0.735836,-0.159327,0,0,0.709955,0.290045
5619,かなしいことよりも くるしいことよりも たのしくてたのしくて しあわせなことよりも おもうことおもうこと,0.540667,-0.198253,0,0,0.676759,0.323241
6081,うへっうへへへっうへっ ランチパック食べちゃうぞ〜〜 うへへへへへ,0.404349,-0.311694,0,0,0.671735,0.328265
2422,わかんねえ…………………天使…………………？？？,0.486669,-0.200479,0,1,0.665332,0.334668
8219,あああああああああああああああああああああああああああああああああああああああああああああああああああああああああああああああああ,0.669800,0.021313,0,1,0.656670,0.343330
1005,可愛いあのこの可愛そうな顔が見たい〜〜〜〜〜〜〜可哀想じゃない子を可愛そがって心底気持ち悪がられたいしあわよくば可愛そうになってくれ〜〜〜〜〜〜〜〜（泣き言）,0.385384,-0.250297,0,0,0.653776,0.346224
3471,でも改めてノワールさんマジマジと見てカッコいい…って泣きそうになってるからほんとノワールさんマジで心の拠り所だよヤバい,0.449262,-0.184845,0,0,0.653420,0.346580
1586,宝石達みんな先生大好きなの可愛いし高い高いされてるの見てキャッキャしちゃう子供らしい部分とか凄い可愛い,0.427555,-0.202361,0,1,0.652471,0.347529


In [ ]:
from IPython.display import HTML
#df_prob_0.head(20)
HTML(df_prob_1.head(10).to_html())

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
735,ラスダンに入ってから実に61周！！！ 間宮、伊良湖は使い切り、ダメコンと女神は１つずつに。 大分苦労させられたものだ。,-0.530041,0.178809,1,0,0.329853,0.670147
4743,地元でヒューガルデン飲める店を開拓しようと3軒あたったが、全てお盆休みだった。,-0.511650,0.140380,1,0,0.342532,0.657468
6946,ひたすらオクタゴン道場 3日目突入、コントローラーの握り方が段々変わってきた 累計118勝17敗,-0.480330,0.153711,1,0,0.346595,0.653405
6053,いや、両部参戦した 15分前に開場してドリンク交換時間的な感じで時間ぴったりに本開場した,-0.498572,0.135448,1,0,0.346600,0.653400
4497,ミュシャ展、20分前には着けるはずだったのに、大江戸線の運休と電車で逆走したせいで未だ到着できず、グヌヌ_(:3｣∠)_ 開場と一緒に入りたかった…,-0.441110,0.180650,1,1,0.349381,0.650619
4188,チケットいただいたので、東京ドームに来た。 ロイヤルウイング初めてだけど、えらく快適。 こりゃ他の席で観れなくなる(&gt;_&lt;),-0.510224,0.107438,1,1,0.350313,0.649687
5839,連休は何も予定を入れてなかったんだけど、 今日の昼は誕生日会、夜はクラスメートと楽しい食事。 良かった、良かった。 そしてもうすでに、エッグハントをやってる＾＾,-0.497502,0.117366,1,0,0.350949,0.649051
2184,"#mastodon (@ サミットストア 三鷹市役所前店 - in 三鷹市, 東京都)",-0.298118,0.313129,1,0,0.351775,0.648225
9753,講究が終わったことに想像以上に安堵していたせいか勢いが余った模様,-0.389374,0.220889,1,1,0.351999,0.648001
466,今日お休みー。ちょっと母のお買い物に付き合ってきた。戦況が全く分からんので、とりあえず敵方の歩兵に喧嘩を売ったらヒラさんだったため、生身ではしゅっと沈められたから、ロボで歩兵を焼いて回ってた。,-0.498846,0.106367,1,0,0.353152,0.646848




---



In [ ]:
accuracy_df = pd.concat([text_df, logits_df, pred_df, label_df, probs_df], axis=1)
# accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
0,#キャプテン翼新アニメにありそうなこと 日向君の雷獣シュートが意図的にピッチを削る「反スポー...,-0.204842,0.054820,1,1,0.435447,0.564553
1,なるほどご本人のツイート辿ったら、リアル友人が、一対一で話すのに、かつご本人がネタバレから入...,-0.081338,0.042540,1,0,0.469070,0.530930
2,不要物の処理をそろそろ始めないとものが溜まってきた。 今日と明日で何とかしなくては…,-0.140710,0.108683,1,1,0.437973,0.562027
3,年末年始とお盆は、母と妹の機嫌がものすごーく悪くなるので、居心地が悪い,-0.140580,0.024364,1,1,0.458857,0.541143
4,風の噂で夜空を見上げる そこには頬をつたう涙の様な星が ふたご座流星群みたいです(^^),0.199045,0.070464,0,0,0.532101,0.467899


In [ ]:
test_df=accuracy_df
list_predLabel=test_df['pred_label'].to_list()
list_trureLabel = test_df['true_label'].to_list()

from sklearn.metrics import accuracy_score
print(accuracy_score(list_trureLabel,  list_predLabel))

from sklearn.metrics import precision_score
print(precision_score(list_trureLabel,  list_predLabel)) #適合率　#適合率（precision）は、陽性と予測されたサンプルのうち正解したサンプルの割合。

from sklearn.metrics import recall_score
print(recall_score(list_trureLabel,  list_predLabel)) #再現率 #再現率（recall）は実際に陽性のサンプルのうち正解したサンプルの割合。


list_predScore = test_df['logit_1']
def sigmoid(x):
    return 1/(1 + np.exp(-x))

list_predScore2=[]
list_predScore2.clear()
for idx, element in enumerate(list_predScore):
  list_predScore2.append(sigmoid(list_predScore[idx]))

# print(list_trureLabel)
# print(list_predScore2)

import numpy as np
from sklearn.metrics import roc_auc_score
roc_auc_score(list_trureLabel,  list_predScore2)

0.4456
0.37268041237113403
0.8105381165919282


0.5345311131771078

In [ ]:
accuracy_df[accuracy_df['pred_label']==0]

,text,logit_0,logit_1,pred_label,true_label,prob_0,prob_1
4,風の噂で夜空を見上げる そこには頬をつたう涙の様な星が ふたご座流星群みたいです(^^),0.199045,0.070464,0,0,0.532101,0.467899
8,「なんつー石頭やねん｣ ゴリラやのーて、雪男ちゃうんか、あんた？,0.049711,0.035260,0,1,0.503613,0.496387
28,ともあれ、エレだとなんかかわいい系おしゃれと同じで、違和感が激しくて手を出しづらいです……,0.052833,0.031744,0,1,0.505272,0.494728
31,ガチャはガチャではない、出るタイミングで課金金額が安いか高いか、それがガチャ運、金を出せばで...,-0.047274,-0.057188,0,0,0.502479,0.497521
34,ツンさんの７つの性格 ツンさんは『清々しい性格』です。 #あなたの７つの性格 Www,0.229457,0.014018,0,0,0.553653,0.446347
...,...,...,...,...,...,...,...
9989,家にいるの暇すぎて出てきてしまった 1時間くらいぶらぶらします,0.050747,0.036416,0,1,0.503583,0.496417
9991,ほんまなんよ さっき暗証番号聞かれて りゅうがので知ったんよあんなこと書いとるの,0.149595,-0.014717,0,0,0.540986,0.459014
9992,このはっきり「春旬」って載ってる圧強アイコンも一緒に拡散されているのだと思うと嬉しくなります...,0.014784,-0.097474,0,0,0.528035,0.471965
9993,いや、他の人は知らない。他の人にはそうは思わない。ジャンジャン使えと思う。 自分に関してはそ...,0.065659,0.037747,0,1,0.506977,0.493023


In [ ]:
# df_text_0= accuracy_df[accuracy_df['pred_label']==0]
list_text_0= df_prob_0.head(1000)['text'].values
# df_text_1= accuracy_df[accuracy_df['pred_label']==1]
list_text_1= df_prob_1.head(1000)['text'].values



---



In [ ]:
!pip install janome
!pip install wordcloud
# !apt-get -y install fonts-ipafont-gothic

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 文章を形態素解析して、Bag of Wordsに変換する
from janome.tokenizer import Tokenizer
import collections
import re

def CountWord(tweet_list):
    all_tweet = "\n".join(tweet_list)

    t = Tokenizer()

    # 原形に変形、名詞のみ、1文字を除去、漢字・平仮名・カタカナの連続のみに限定
    c = collections.Counter(token.base_form for token in tqdm(t.tokenize(all_tweet))
                            if token.part_of_speech.startswith('名詞') and len(token.base_form) > 1 
                            and token.base_form.isalpha() and not re.match('^[a-zA-Z]+$', token.base_form)) 
    # for token in tqdm(t.tokenize(all_tweet)):
    #   if token.part_of_speech.startswith('名詞') and len(token.base_form) > 1 and token.base_form.isalpha() and not re.match('^[a-zA-Z]+$', token.base_form):
    #     c = collections.Counter(token.base_form)

    pre_dict_freq = {}
    mc = c.most_common()
    for elem in mc:
        pre_dict_freq[elem[0]] = elem[1]

    list_freq=list(pre_dict_freq)
    #print(list_c)
    #print(c)
    for idx, ch in enumerate(list_freq[:100]):
        if not (ch < u'ぁ' or ch > u'ん') and ch != '_': #ひらがなの場合
          #print(ch)
          del pre_dict_freq["%s" % ch]
        else:
          pass
    #print() 

    dict_freq=pre_dict_freq
    return dict_freq


dict_word_freq_0_A = CountWord(list_text_0)
dict_word_freq_1_A = CountWord(list_text_1)
# print(dict_word_freq_0_A)
# print()
# print(dict_word_freq_1_A)
# print()

In [ ]:
print(TYPE_INDEX)
dict_word_freq_0_B={}
for key, value in dict_word_freq_0_A.items():
    if value >= 10:
        dict_word_freq_0_B[key] = value

print(dict_word_freq_0_B)
print()

dict_word_freq_1_B={}
for key, value in dict_word_freq_1_A.items():
    if value >= 10:
        dict_word_freq_1_B[key] = value

print(dict_word_freq_1_B)
print()

JorP_J
{'前日': 270, '好き': 80, '自分': 76, 'ツイート': 62, 'リプライ': 54, '気温': 53, 'フォロー': 49, '最高': 44, '殺意': 33, '天気': 31, '感じ': 29, '最近': 28, '言葉': 25, '明日': 25, '最低': 25, '時間': 25, '今日': 24, '今年': 24, '気持ち': 22, '世界': 21, '素敵': 21, '幸せ': 19, '来年': 19, '大好き': 18, '人間': 17, 'ーーー': 15, '仕事': 14, '女性': 14, 'アニメ': 14, '意味': 13, '女子': 13, '最後': 13, 'ゲーム': 13, '写真': 12, 'スモ': 12, 'ーー': 11, '一緒': 11, '大切': 11, 'キャラ': 11, '友達': 11, '無理': 11, '以外': 11, '相手': 11, '周り': 11, '一番': 11, '結果': 11, '昨日': 11, 'ランク': 11, 'マジ': 10, 'エロ': 10}

{'今日': 70, '時間': 47, '無料': 39, '仕事': 37, 'マンガ': 36, '明日': 28, '自分': 27, '質問': 25, '昨日': 22, '感じ': 17, '開始': 17, 'ラーメン': 17, '東京': 16, '生放送': 15, '途中': 15, '予約': 15, '予定': 14, 'ニュース': 14, '部屋': 14, '楽しみ': 14, '大丈夫': 14, '移動': 14, '更新': 13, '場所': 13, '参加': 13, '購入': 13, '最近': 13, '以上': 12, 'イベント': 12, 'レベル': 12, '追加': 12, '結果': 12, '終了': 12, '結局': 12, '全部': 12, '登録': 12, 'ゲーム': 12, '電車': 11, '来年': 11, '状態': 11, '関係': 11, '試験': 10, '病院': 10, '職場': 10, '週間': 10}



In [ ]:
# df_word_freq_0 = pd.DataFrame(dict_word_freq_0_B.values(), index=dict_word_freq_0_B.keys())
# HTML(df_word_freq_0.to_html())


In [ ]:
# df_word_freq_1 = pd.DataFrame(dict_word_freq_1_B.values(), index=dict_word_freq_1_B.keys())
# HTML(df_word_freq_1.to_html())

In [ ]:
# dict_word_freq_0_B[list(dict_word_freq_0_B)[0]]

In [ ]:
print(TYPE_INDEX)
max_freq_0=dict_word_freq_0_B[list(dict_word_freq_0_B)[0]]
list_words_0=[]
list_idx_0=[]
for i in range(0, max_freq_0+1):
    list_words_0.append(np.nan)
    list_idx_0.append(i+1)
# print(len(list_words_0))
for i in range(max_freq_0,0,-1):
    list_sub=[]
    #print(i)
    for Words,freq in dict_word_freq_0_B.items():
        if freq==i:
            #print(freq, Words)
            list_sub.append(Words)
    #print(list_sub)
    list_words_0[i]=list_sub
    #print(list_words_0)    
# print(len(list_words_0))
# print(len(list_idx))

dict_word_freq_0_C={}
for j in range(max_freq_0,0,-1):
    DDD=str(list_words_0[j])
#     print(DDD)
    DDD=DDD.replace('[', '')
    DDD=DDD.replace(']', '')
    DDD=DDD.replace('\'', '')
#     print(DDD)
    dict_word_freq_0_C['%s' % DDD]= list_idx_0[j-1]

df_word_freq_0 = pd.DataFrame(dict_word_freq_0_C.values(), index=dict_word_freq_0_C.keys())
HTML(df_word_freq_0.to_html())

JorP_J


,0
前日,270
,1
好き,80
自分,76
ツイート,62
リプライ,54
気温,53
フォロー,49
最高,44
殺意,33


In [ ]:
print(TYPE_INDEX)
max_freq_1=dict_word_freq_1_B[list(dict_word_freq_1_B)[0]]
list_words_1=[]
list_idx_1=[]
for i in range(0, max_freq_1+1):
    list_words_1.append(np.nan)
    list_idx_1.append(i+1)
# print(len(list_words_0))
for i in range(max_freq_1,0,-1):
    list_sub=[]
    #print(i)
    for Words,freq in dict_word_freq_1_B.items():
        if freq==i:
            #print(freq, Words)
            list_sub.append(Words)
    #print(list_sub)
    list_words_1[i]=list_sub
    #print(list_words_0)    
# print(len(list_words_0))
# print(len(list_idx))

dict_word_freq_1_C={}
for j in range(max_freq_1,0,-1):
    DDD=str(list_words_1[j])
#     print(DDD)
    DDD=DDD.replace('[', '')
    DDD=DDD.replace(']', '')
    DDD=DDD.replace('\'', '')
#     print(DDD)
    dict_word_freq_1_C['%s' % DDD]= list_idx_1[j-1]

df_word_freq_1 = pd.DataFrame(dict_word_freq_1_C.values(), index=dict_word_freq_1_C.keys())
HTML(df_word_freq_1.to_html())

JorP_J


,0
今日,70
,1
時間,47
無料,39
仕事,37
マンガ,36
明日,28
自分,27
質問,25
昨日,22


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from IPython.display import HTML

def words_freq(corpus):
    pd.set_option('display.max_columns', None)
    pd.options.display.float_format = '{:0.2f}'.format

    count_vectorizer = CountVectorizer()
    bow = count_vectorizer.fit_transform(corpus).toarray()
    #------------------------------------------------------------------------------------
    # --- BoW (Bag of Words) ---
    #------------------------------------------------------------------------------------

    print('--- BoW (Bag of Words) ---')
    try:
        df = pd.DataFrame(bow,columns=count_vectorizer.get_feature_names())
    except:
        df = pd.DataFrame(bow,columns=count_vectorizer.get_feature_names_out())
    #------------------------------------------------------------------------------------
    # TF を計算してるところ (行方向の処理)
    #------------------------------------------------------------------------------------
    # 文書に含まれる単語の数をカウントする
    number_of_words = np.sum(bow, axis=1, keepdims=True)
    # 文書の中での単語の頻度を計算する
    tf = bow / number_of_words
    try:
        df_tf = pd.DataFrame(tf,columns=count_vectorizer.get_feature_names())
    except:
        df_tf = pd.DataFrame(tf,columns=count_vectorizer.get_feature_names_out())

    #------------------------------------------------------------------------------------
    # IDF を計算してるところ (列方向の処理)
    #------------------------------------------------------------------------------------
    #print('--- IDF (Inverse Document Frequency) ---')
    # 文書の数をカウントする
    number_of_docs = len(corpus)
    # その単語が一つでも含まれる文書の数をカウントする
    number_of_docs_contain_word = np.count_nonzero(bow, axis=0)
    # 単語の珍しさを計算する
    idf = np.log(number_of_docs / number_of_docs_contain_word)
    try:
        df_idf = pd.DataFrame([idf],columns=count_vectorizer.get_feature_names())
    except:
        df_idf = pd.DataFrame([idf],columns=count_vectorizer.get_feature_names_out())
    #------------------------------------------------------------------------------------
    # TF-IDF を計算してるところ
    #------------------------------------------------------------------------------------
    #print('--- TF-IDF ---')
    # TF と IDF をかける
    #
    tfidf = tf * idf
    try:
        df_tfidf = pd.DataFrame(tfidf,columns=count_vectorizer.get_feature_names())
    except:
        df_tfidf = pd.DataFrame(tfidf,columns=count_vectorizer.get_feature_names_out())
    # df_results=pd.DataFrame(df.sum(), columns=["words", "sum_TF-IDF"])
    df_results=pd.DataFrame( columns=["words", "sum_TF-IDF"])
    df_results["words"]=df_tfidf.columns.to_list()
    df_results["sum_TF-IDF"]=df_tfidf.sum().to_list()
    df_results=df_results.set_index("words")
    df_results.sort_values("sum_TF-IDF", ascending=False, inplace=True)

    return df_results

# corpus = [
#           'This is the first document.',
#           'This document is the second document.',
#           'And this is the third one.',
#           'Is this the first document?',
#           ]

# df_results=words_freq(list_text_0)
# HTML(df_results.to_html())

In [ ]:
list_sentence_ifidf_0=[]
list_sentence_ifidf_0.clear()

for sent in list_text_0:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    one_sentence=' '.join(token_words)
    list_sentence_ifidf_0.append(one_sentence)
    
df_tfidf_0=words_freq(list_sentence_ifidf_0)
HTML(df_tfidf_0[:100].to_html())

--- BoW (Bag of Words) ---


/home/20VR016F/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


,sum_TF-IDF
words,
unk,66.66
前日,55.09
ない,30.81
いい,26.50
です,26.40
たい,25.25
って,23.82
てる,23.54
さん,21.88


In [ ]:
list_sentence_ifidf_1=[]
list_sentence_ifidf_1.clear()
print(len(list_text_1))
for sent in list_text_1:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    one_sentence=' '.join(token_words)
    list_sentence_ifidf_1.append(one_sentence)
    
df_tfidf_1=words_freq(list_sentence_ifidf_1)
HTML(df_tfidf_1[:100].to_html())

1000
--- BoW (Bag of Words) ---


/home/20VR016F/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,sum_TF-IDF
words,
unk,48.12
ない,22.58
まし,22.49
です,21.55
から,20.87
けど,19.39
ます,17.05
ので,15.39
てる,14.51


In [ ]:
# # Word Cloudで可視化、WordCloud可視化関数
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties

# def make_cloud(dict_word_freq, name_16type):
#   print(name_16type)
#   print(dict_word_freq)
#   fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'  # 日本語フォント指定

#   stop_words = ['さん','そう'] 
#   wordcloud = WordCloud(
#       font_path=fpath,
#       width=900, height=600,   # default width=400, height=200
#       background_color="white",   # default=”black”
#       stopwords=set(stop_words),
#       max_words=500,   # default=200
#       min_font_size=4,   #default=4
#       collocations = False   #default = True
#       ).fit_words(dict_word_freq) #https://water2litter.net/rum/post/python_wordcloud_fit/
  
#   plt.figure(figsize=(15,12))
#   plt.imshow(wordcloud)
#   plt.axis("off")
#   plt.savefig("word_cloud_%s.png" %  name_16type)
#   plt.show()

#   print()
# make_cloud(dict_word_freq_0, "I")
# make_cloud(dict_word_freq_1, "E")
# #DrawWordCloud(dict_word_freq_ENFJ , "ENFJ")